In [62]:
import csv
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from sklearn.preprocessing import LabelEncoder

In [94]:
# read train.csv and prepare train/test data set out of it
data = pd.read_csv("./train.csv")
data.pop('id') # drop id

# extract integer encoded label from species
species = data.pop('species')
data['labels'] = LabelEncoder().fit(species).transform(species)

data.head(1)

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,labels
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.0,0.00293,0.00293,0.035156,0.0,0.0,0.004883,0.0,0.025391,3
